In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일 자동으로 루트에서 찾아서 로드
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings

# PDF 로딩
loader = PyPDFLoader("../data/sleep_performance.pdf")
documents = loader.load_and_split()

# 임베딩 생성
embedding = OpenAIEmbeddings()  # .env에 API 키 필요!
vectorstore = FAISS.from_documents(documents, embedding)

# 저장
vectorstore.save_local("vectorstores/sleep_rag")


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26292\2527867866.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()  # .env에 API 키 필요!


In [ ]:
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 벡터 DB 로딩
vectorstore = FAISS.load_local(
    "vectorstores/sleep_rag",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# LLM 설정
llm = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG 체인 생성
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 질의 실행
query = "해당 논문을 정리해서 설명해줘"
response = rag_chain(query)
print(response["result"])
